# 0. Install and Import Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

     |███████████████████████▍        | 287.8 MB 94.5 MB/s eta 0:00:022   |██████                          | 73.8 MB 17.1 MB/s eta 0:00:19

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |██████████████████████████████▉ | 379.9 MB 103.4 MB/s eta 0:00:01

     |███████████████████████████████▊| 391.3 MB 103.4 MB/s eta 0:00:01

     |████████████████████████████████| 394.4 MB 103.4 MB/s eta 0:00:01

     |████████████████████████████████| 394.4 MB 9.2 kB/s 
     |███████████████████████▍        | 288.5 MB 100.3 MB/s eta 0:00:02   |███████▊                        | 94.9 MB 34.6 MB/s eta 0:00:09

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████████▉| 391.9 MB 107.0 MB/s eta 0:00:01

     |████████████████████████████████| 394.4 MB 107.0 MB/s eta 0:00:01

     |████████████████████████████████| 394.4 MB 7.4 kB/s 
     |████████████████████████████████| 60.4 MB 5.5 kB/s  eta 0:00:01     |████████████████████████▍       | 46.0 MB 61.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 71.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.7 MB/s  eta 0:00:01
     |████████████████████████████████| 1.1 MB 71.0 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 5.8 MB 61.1 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.3 MB/s  eta 0:00:01
     |████████████████████████████████| 462 kB 71.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 66.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 67.0 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 71.9 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 1.9 MB/s  eta 0:00:01
  Using cached Werkzeug-2.0.2-py3-none-any.whl

In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

2022-01-01 02:42:46.331466: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-01 02:42:46.331492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
!wget -q -O lite-model_movenet_singlepose_lightning_3.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/3?lite-format=tflite

# 1. Load Model

In [6]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

# 2. Make Detections

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 3. Draw Keypoints

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

# 4. Draw Edges

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)